In [1]:
%run ../../../utils/commonImports.py
%run ../../../utils/tradingImports.py
%matplotlib inline

from itertools import product
from base import *
from exputils import *

# Load data

In [4]:
# dropbox_dir = 'D:\\Dropbox\\My work\\krypl-project'
dropbox_dir = '/Users/david.vesely/Dropbox/My work/krypl-project'
features = ['change_1', 'change_2', 'change_3']
pairs = read_tsv('selected-pairs.tsv', header=None)[0].tolist()

val_dict = {p: read_val(p, dropbox_dir) for p in pairs}
ohlc_featured_train = load_model('ohlc_featured_val.pkl')

In [5]:
model = load_model('models/lin_regression.pkl')

In [7]:
results = pd.read_pickle('train_results.pkl')

# Setup

In [6]:
features = ['support', 'change_1', 'change_2', 'change_3']

window_sizes = [4]
shifts = [1, 2, 3]

DAY = 48
max_hold_time = 5 * DAY
stop_loss = 0.05
fee = 0.0025

min_profit_targets = [round(i / 100, 2) for i in range(0, 6)]

args = product(pairs, window_sizes, min_profit_targets)
args = list(args)
len(args)

114

# Select params

In [9]:
def min_max(x):
    return (x - x.min()) / (x.max() - x.min())

In [10]:
results['zTotalProfit'] = min_max(results['totalProfit'])
results['zWinPercentage'] = min_max(results['winPercentage'])
results['zAvgLossTrade'] = min_max(results['avgLossTrade'])
results['meanStats'] = (results['zTotalProfit'] + 
                        (2 * results['zWinPercentage']) + 
                        results['zAvgLossTrade'] * (1 - results['winPercentage'] / 100)) / 3

In [30]:
selected_params = pd.DataFrame()
for pair in pairs:
    p_results = results[results['pair'] == pair]
    p_results = reset_index_hard(p_results).copy()
    row = p_results.query('totalProfit > 0')\
        .query('-avgLossTrade <= avgWinTrade')\
        .sort_values('totalProfit', ascending=False)
    if len(row) > 0:
        selected_params = selected_params.append(row.iloc[0, :])

In [31]:
cols = ['pair', 'window_size', 'stop_loss', 'min_profit_target', 
         'totalProfit', 'maxTimeToClose', 'winPercentage', 'avgLossTrade', 'avgWinTrade']
selected_params[cols]

,pair,window_size,stop_loss,min_profit_target,totalProfit,maxTimeToClose,winPercentage,avgLossTrade,avgWinTrade
2,BTC_ETH,4.0000,0.0500,0.0500,258.7193,186.5000,66.6667,-4.5107,7.4297
4,BTC_XRP,4.0000,0.0500,0.0400,36.0522,408.2500,56.7568,-4.5107,5.1535
4,BTC_LTC,4.0000,0.0500,0.0500,3.5220,141.5000,50.0000,-4.5107,8.0328
5,BTC_XMR,4.0000,0.0500,0.0400,151.9209,289.7500,58.2609,-4.5107,5.4990
5,BTC_STR,4.0000,0.0500,0.0500,98.5593,181.0000,61.3636,-4.5107,6.4904
2,BTC_DASH,4.0000,0.0500,0.0400,126.8735,178.2500,69.4915,-4.5107,5.0748
0,BTC_BTS,4.0000,0.0500,0.0300,64.6783,601.7500,56.1905,-4.4974,4.6027
1,BTC_FCT,4.0000,0.0500,0.0300,335.5430,275.5000,62.5000,-4.5107,5.1032
2,BTC_XEM,4.0000,0.0500,0.0300,249.3750,163.0000,64.9682,-4.5107,4.8771
0,BTC_MAID,4.0000,0.0500,0.0300,403.7325,372.0000,66.9811,-4.5107,5.0668
